# Traffic data aggregation using network distance metrics and cokriging
Final project for GEOG 8102  
Yue Lin (lin.3326 at osu.edu)  
Created: 4/14/2020

## Mapping points to non-Euclidean space

Read graph data

In [15]:
import csv
with open('d04_tomtom_od.csv', newline='') as csvfile:
    alllines = csv.reader(csvfile)
    graph = []
    for line in alllines:
        item = []
        item.append(int(line[2]))
        item.append(int(line[3]))
        item.append(float(line[5]))
        graph.append(item)
    print(graph[:5])

[[1, 1, 0.0], [1, 220, 3145.43243], [1, 290, 3189.500113], [1, 221, 6100.009007], [1, 328, 10813.73763]]


Costruct distance matrix

In [16]:
## number of nodes
n = max([g[1] for g in graph])

## distance matrix
INF = float(0)
distance_matrix = [ [INF] * n for i in range(n)]
for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    distance_matrix[i][j] = d
    distance_matrix[j][i] = d
# print("distance matrix:", distance_matrix[0])

Construct b matrix

In [17]:
# extract a column
def column(matrix, i):
    return [row[i] for row in matrix]


b_matrix = [ [INF] * n for i in range(n)]

d_sum = 0
for i in range(n):
    for j in range(n):
        d = distance_matrix[i][j]
        d_sum = d_sum + d*d
       
for i in range(n):
    for j in range(n):
        d = distance_matrix[i][j]
        
        d_x = distance_matrix[i]
        d_x_sum = 0
        for k in range(n):
            d = distance_matrix[i][k]
            d_x_sum = d_x_sum + d*d

        d_y = column(distance_matrix, j)
        d_y_sum = 0
        for k in range(n):
            d = distance_matrix[k][j]
            d_y_sum = d_y_sum + d*d
        
        d = distance_matrix[i][j]
        
        b_matrix[i][j] = 1/2 * (-d*d + 1/n * d_x_sum + 1/n * d_y_sum - 1/(n*n) * d_sum)

# print("b matrix:", b_matrix[0])

Get the new point set

In [28]:
import numpy as np
import scipy.linalg as la

B = np.array(b_matrix)
eigvals, eigvecs = la.eig(B)
sorted_eigvals = sorted(eigvals, reverse = True)

sorted_eigvals = np.array(sorted_eigvals[:2])
V = np.diag(sorted_eigvals**(0.5))
print(V.shape)

eigvals = eigvals.tolist()
index1 = eigvals.index(sorted_eigvals[0])
Q1 = np.array([eigvecs[index1]])
index2 = eigvals.index(sorted_eigvals[1])
Q2 = np.array([eigvecs[index2]])
Q = np.concatenate((Q1, Q2), axis=0)
print(Q.shape)

S_new = np.dot(np.transpose(Q), V)
S_new = S_new.real
print(S_new.dtype)
print(S_new[0][0])

(2, 2)
(2, 418)
float64
36736.91777467165


Write new coordinates to file

In [30]:
import csv

with open('d04_tomtom.csv', 'r') as read_obj, \
        open('d04_tomtom_1.csv', 'w', newline='') as write_obj:
    csv_reader = csv.reader(read_obj)
    csv_writer = csv.writer(write_obj)
    
    i = 0
    for row in csv_reader:
        row.append(S_new[i][0])
        row.append(S_new[i][1])
        i = i + 1
        
        csv_writer.writerow(row)